In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

### Rough Work

In [6]:
D = 10
means = [np.repeat(0, D).reshape(-1), np.repeat(1, D).reshape(-1)]
covariance = np.identity(D)
priors = [0.5, 0.5]
train_size, test_size = 2000, 1000

In [7]:
X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance, size=int(train_size*priors[1]))
X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance, size=int(test_size*priors[1]))
X_train = np.vstack((X_train_0, X_train_1))
X_test = np.vstack((X_test_0, X_test_1))

y_train = np.vstack((
    np.repeat(0, int(train_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(train_size*priors[1])).reshape(-1, 1)
))
y_test = np.vstack((
    np.repeat(0, int(test_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(test_size*priors[1])).reshape(-1, 1)
))

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2000, 10), (2000, 1), (1000, 10), (1000, 1))

In [ ]:
M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.inv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

In [ ]:
def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0]
upper = np.matmul(W.T, M_1)[0]
for b in np.linspace(lower-1, upper+1, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Best solution acc. =', best_acc/X_train.shape[0])

Bias -1.014448619774113, Acc: 0.918, Best: 0.918
Bias -1.0093357597719812, Acc: 0.918, Best: 0.918
Bias -1.0042228997698495, Acc: 0.918, Best: 0.918
Bias -0.9991100397677177, Acc: 0.9175, Best: 0.918
Bias -0.993997179765586, Acc: 0.9165, Best: 0.918
Bias -0.9888843197634543, Acc: 0.915, Best: 0.918
Bias -0.9837714597613225, Acc: 0.915, Best: 0.918
Bias -0.9786585997591908, Acc: 0.9135, Best: 0.918
Bias -0.973545739757059, Acc: 0.9115, Best: 0.918
Bias -0.9684328797549273, Acc: 0.9105, Best: 0.918
Bias -0.9633200197527956, Acc: 0.91, Best: 0.918
Bias -0.9582071597506638, Acc: 0.91, Best: 0.918
Bias -0.9530942997485321, Acc: 0.9095, Best: 0.918
Bias -0.9479814397464004, Acc: 0.909, Best: 0.918
Bias -0.9428685797442686, Acc: 0.91, Best: 0.918
Bias -0.9377557197421369, Acc: 0.91, Best: 0.918
Bias -0.9326428597400052, Acc: 0.9085, Best: 0.918
Bias -0.9275299997378734, Acc: 0.909, Best: 0.918
Bias -0.9224171397357417, Acc: 0.909, Best: 0.918
Bias -0.91730427973361, Acc: 0.9085, Best: 0.918
B

In [ ]:
acc = 0
b = best_b
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1
    
print(f'Test acc. = {acc/X_test.shape[0]}')

Test acc. = 0.919


Different from FLDA ?

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train.reshape(-1))
lda.score(X_train, y_train.reshape(-1)), lda.score(X_test, y_test.reshape(-1))

(0.9435, 0.948)

### FLDA for 10-D

In [8]:
train_acc, test_acc = [], []
prior_0 = []
p = 0.5
D = 2
means = [np.repeat(0, D).reshape(-1), np.repeat(1, D).reshape(-1)]
covariance = np.identity(D)
priors = [p, 1-p]
train_size, test_size = 2000, 1000

X_train_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(train_size*priors[0]))
X_train_1 = np.random.multivariate_normal(mean=means[1], cov=covariance, size=int(train_size*priors[1]))
X_test_0 = np.random.multivariate_normal(mean=means[0], cov=covariance, size=int(test_size*priors[0]))
X_test_1 = np.random.multivariate_normal(mean=means[1], cov=covariance, size=int(test_size*priors[1]))
X_train = np.vstack((X_train_0, X_train_1))
X_test = np.vstack((X_test_0, X_test_1))

y_train = np.vstack((
    np.repeat(0, int(train_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(train_size*priors[1])).reshape(-1, 1)
))
y_test = np.vstack((
    np.repeat(0, int(test_size*priors[0])).reshape(-1, 1),
    np.repeat(1, int(test_size*priors[1])).reshape(-1, 1)
))

M_0 = X_train_0.mean(axis=0)
M_1 = X_train_1.mean(axis=0)

cov_0 = np.zeros((D, D))
for i in range(X_train_0.shape[0]):
    W = (X_train_0[i] - M_0).reshape(-1, 1)
    cov_0 += np.matmul(W, W.T)

cov_1 = np.zeros((D, D))
for i in range(X_train_1.shape[0]):
    W = (X_train_1[i] - M_1).reshape(-1, 1)
    cov_1 += np.matmul(W, W.T)

S_w = cov_0 + cov_1
S_w_inv = np.linalg.pinv(S_w)
W = np.matmul(S_w_inv, (M_1 - M_0)).reshape(-1, 1)
W = W/np.linalg.norm(W,2)

def predict(x, W, b):
    return np.matmul(W.T, x) + b


# Line Search
best_b, best_acc = 0, 0
lower = np.matmul(W.T, M_0)[0] - 5
upper = np.matmul(W.T, M_1)[0] + 5
for b in np.linspace(lower, upper, 1000):
    acc = 0
    for (i, X_i) in enumerate(X_train_0):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred < 0:
            acc += 1

    for (i, X_i) in enumerate(X_train_1):
        pred = predict(X_i.reshape(-1, 1), W, b)
        if pred >= 0:
            acc += 1

    if acc > best_acc:
        best_acc = acc
        best_b = b
    
    # print(f'Bias {b}, Acc: {acc/X_train.shape[0]}, Best: {best_acc/X_train.shape[0]}')

print('Train acc. =', best_acc/X_train.shape[0])

b = best_b
acc = 0
for (i, X_i) in enumerate(X_test_0):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        acc += 1

for (i, X_i) in enumerate(X_test_1):
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred >= 0:
        acc += 1

print(f'Test acc. = {acc/X_test.shape[0]}')

# f1-score
y_pred = []
for X_i in X_train:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)
print('F1-Score (train) = ', f1_score(y_train, y_pred))

y_pred = []
for X_i in X_test:
    pred = predict(X_i.reshape(-1, 1), W, b)
    if pred < 0:
        y_pred.append(0)
    else:
        y_pred.append(1)
y_pred = np.array(y_pred)

print('F1-Score (test) = ', f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Train acc. = 0.7575
Test acc. = 0.742
F1-Score (train) =  0.7554210791729702
F1-Score (test) =  0.7393939393939394


array([[376, 124],
       [134, 366]])